# Inverse problem for the wave equation using an operator recurrent neural network

We consider the inverse problem to find $a$ in the below wave equation given 
the Neumann-to-Dirichlet map 

$$
\Lambda h = u|_{x = 0},
$$

where $u$ is the solution to the problem 

$$
\begin{cases}
\partial_t^2 u - a(x) \partial_x^2 u = 0, & \text{on $(0,T) \times (0,L)$},
\\
\partial_x u|_{x=0} = h, \quad \partial_x u|_{x=L} = 0,
\\
u|_{t=0} = 0, \quad \partial_t u|_{t=0} = 0.
\end{cases}
$$

Here we consider only a subproblem related to the inverse problem to find $a$.
In Section 2 of 

> Jussi Korpela, Matti Lassas and Lauri Oksanen.
> _Regularization strategy for an inverse problem for a 1 + 1 dimensional wave equation_.
> Inverse Problems 32, 065001, 2016.
> <https://doi.org/10.1088/0266-5611/32/6/065001> 

it was shown that $\Lambda$ determines the following volumes 

$$
V(r) = \int_0^{\chi(r)} \frac{1}{c(x)^2} dx
$$

and that these volumes then determine $a$.
Here $c^2 = a$ and $\chi$ is the inverse function of $\tau$ defined by

$$
\tau(y) = \int_0^y \frac{1}{c(x)} dx.
$$


We consider the subproblem to compute a single volume $V(r_0)$, with fixed $r_0>0$, given $\Lambda$. 
We solve this problem using a neural network, with the network architecture taken from 

> Maarten V. de Hoop, Matti Lassas, Christopher A. Wong. _Deep learning architectures for nonlinear operator functions and nonlinear inverse problems_. [arXiv:1912.11090](https://arxiv.org/abs/1912.11090)

The training data consists of pairs $(\Lambda, V(r_0))$ corresponding to different functions $a$.
Here $\Lambda$ is, of course, discretized, and the details of the discretization are discussed in the notebook describing the generation of the data. 


# Initialization

In [121]:
import numpy as np
import torch

import opnet
from volume_inversion_data import generate_data, save_data, load_data

#PATH = './volume_inversion_ReLU3.pth' #biggest data (10000/2500)
#PATH = './volume_inversion_NOReLU3.pth' #biggest data (10000/2500)

#PATH = './volume_inversion_NOReLU2.pth' #big data (6000/1000)
#PATH = './volume_inversion_ReLU2.pth' #big data (6000/1000)

#PATH = './volume_inversion_NOReLU.pth' #normal data (600/100)
#PATH = './volume_inversion_ReLU.pth' #normal data (600/100)

#PATH = './volume_inversion_netLONG.pth' # trained 1600 epochs

PATH = './volume_inversion_net.pth' #that network that can be overwritten

Specify the network model and the loss function.

In [122]:

dim = 126 # this needs be the size of Lambda_h
num_layers = 5
# luodaan uusi neuroverkko
model = opnet.OperatorNet(dim, 2*num_layers, scalar_output=True, useReLU=False)
#model = opnet.OperatorNet(dim, num_layers, scalar_output=True, useReLU=True)
loss_fn = torch.nn.MSELoss()

# Generation of training data

In [123]:
#save_data(*generate_data(1000), "volume_inversion_train_data1000.npz")
#save_data(*generate_data(350), "volume_inversion_test_data350.npz")

train_data_path = "volume_inversion_train_dataBIG.npz"
test_data_path = "volume_inversion_test_dataBIG.npz"

# Training

In [124]:
import wave_training_and_testing
import os.path

# update changes
from importlib import reload 
reload(wave_training_and_testing)
reload(opnet)

lr=1e-4

# upload the neural network used earlier:
# if os.path.exists(PATH):
#     # PATH-tiedostoon on tallennettu verkko ja tämä lataa sen kertoimet ylempänä 
#     # luotuun verkkoon, eli jatketaan vanhalla verkolla
#     model.load_state_dict(torch.load(PATH)) 
# else:
#     print("no old PATH, I'll make a new one")

wave_training_and_testing.wave_training_and_testing(model, loss_fn, lr, PATH, train_data_path, test_data_path)


20 0.021636
40 0.009329
60 0.006151
80 0.004480
100 0.003563
120 0.003026
140 0.002676
160 0.002423
180 0.002223
200 0.002057
220 0.001916
240 0.001792
260 0.001684
280 0.001588
300 0.001502
320 0.001425
340 0.001356
360 0.001294
380 0.001238
400 0.001187
420 0.001141
440 0.001100
460 0.001062
480 0.001027
500 0.000995
520 0.000966
540 0.000939
560 0.000914
580 0.000891
600 0.000869
620 0.000849
640 0.000830
660 0.000812
680 0.000796
700 0.000780
720 0.000765
740 0.000751
760 0.000738
780 0.000725
800 0.000713
820 0.000702
840 0.000691
860 0.000680
880 0.000670
900 0.000661
920 0.000651
940 0.000642
960 0.000634
980 0.000625
1000 0.000617
1020 0.000610
1040 0.000602
1060 0.000595
1080 0.000588
1100 0.000581
1120 0.000574
1140 0.000568
1160 0.000562
1180 0.000556
1200 0.000550
1220 0.000544
1240 0.000538
1260 0.000533
1280 0.000528
1300 0.000522
1320 0.000517
1340 0.000513
1360 0.000508
1380 0.000503
1400 0.000498
1420 0.000494
1440 0.000490
1460 0.000485
1480 0.000481
1500 0.000477
152

Choose the optimization method.

In [125]:
# Learning rate parameter is from the quickstart guide 
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Loop over the training data multiple times (epochs) and 
save the optimized parameters. 

In [126]:
# for epoch in range(2): 
#     print(f"Epoch {epoch+1}\n-------------------------------")
#     for batch, (X, y) in enumerate(train_loader):
#         # Compute prediction error
#         pred = model(X)
#         loss = loss_fn(pred, y)
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         # Print statistics
#         if batch % 10 == 0:
#             n, N = (batch + 1) * len(X), len(train_loader.dataset)
#             print(f"loss: {loss.item():>7f}  [{n:>5d}/{N:>5d}]")

torch.save(model.state_dict(), PATH)

# Testing

If we have already trained the network, we can just load its parameters. (Note that we still need to run the initialization.)

In [127]:
## Load trained variables
# model.load_state_dict(torch.load(PATH))

In [128]:
# # Load the testing data
# test_loader = torch.utils.data.DataLoader(
#     load_data("volume_inversion_test_data.npz"), 
#     batch_size=64)

Compute a couple of samples.

In [129]:
# dataiter = iter(test_loader)
# X, y = dataiter.next()
# with torch.no_grad():
#     pred = model(X)
# print("True: ")
# print(y[:2])
# print("Prediction: ")
# print(pred[:2])

In [130]:
# num_batches = len(test_loader)
# test_loss = 0
# with torch.no_grad():
#     for X, y in test_loader:
#         pred = model(X)
#         test_loss += loss_fn(pred, y).item()
# test_loss /= num_batches
# print(f"Avg loss: {test_loss:>8f} \n")